In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataframe/df500k.pkl
/kaggle/input/dataframe/yelp_stopwords(3m).txt
/kaggle/input/dataframe/eng_w2v2
/kaggle/input/dataframe/yelp_stopwords(500k).txt
/kaggle/input/dataframe/m.model
/kaggle/input/dataframe/m.vocab
/kaggle/input/dataframe/df3m.pkl


In [2]:
df = pd.read_pickle("/kaggle/input/dataframe/df500k.pkl")

In [3]:
X=[]
for line in df['text']:
    X.append(line)

from keras.preprocessing.text import Tokenizer

t= Tokenizer()
t.fit_on_texts(X)
count=t.word_counts
print(len(count))

sorted_count=sorted(count.items(), key=lambda item: item[1], reverse=True)
word_data=pd.DataFrame(sorted_count)
word_data= word_data.rename(columns={0: "key", 1:"value"})
x=word_data[0:50]['key']
y=word_data[0:50]['value']
word_data
    

Using TensorFlow backend.


193441


,key,value
0,the,2997391
1,and,2019486
2,i,1681123
3,to,1589470
4,a,1502952
...,...,...
193436,vulgarized,1
193437,pyrus,1
193438,spaccavento,1
193439,ecg,1


In [4]:
#단어 빈도수 분포 
y2_l=len(word_data)
list1=[]
list2=[]
index=0
for i in range(y2_l): 
    cnt=0
    while y2_l-index-1>=0 and i+1==word_data['value'][y2_l-index-1]:
        cnt+=1
        #print(y2_l-index-1,y2[y2_l-index-1],cnt)
        index+=1
    if(cnt!=0):
        list1.append(i+1)
        list2.append(cnt) 

In [5]:
word_count_data = pd.DataFrame(list(zip(list1, list2)), 
               columns =['count', 'ammount']) 
word_count_data[:100]

,count,ammount
0,1,94092
1,2,23605
2,3,11541
3,4,7302
4,5,5208
...,...,...
95,96,74
96,97,73
97,98,65
98,99,65


In [6]:
total=0
for i in range(100):
    total+=word_count_data['ammount'][i]
print(total,total/y2_l*100)

180371 93.24341788969247


In [7]:
import sentencepiece as spm
spm.SentencePieceTrainer.train('--input=/kaggle/input/dataframe/yelp_stopwords(500k).txt --model_prefix=m --vocab_size=10000')
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [8]:
#import sentencepiece as spm
#sp = spm.SentencePieceProcessor()
#sp.load('/kaggle/input/dataframe/m.model')
X2=[]
for i, line in enumerate(df['text']):
    if i%1000==0:
        print(i,end='')
    X2.append(sp.encode_as_pieces(line))

print(X2[:5])

0100020003000400050006000700080009000100001100012000130001400015000160001700018000190002000021000220002300024000250002600027000280002900030000310003200033000340003500036000370003800039000400004100042000430004400045000460004700048000490005000051000520005300054000550005600057000580005900060000610006200063000640006500066000670006800069000700007100072000730007400075000760007700078000790008000081000820008300084000850008600087000880008900090000910009200093000940009500096000970009800099000100000101000102000103000104000105000106000107000108000109000110000111000112000113000114000115000116000117000118000119000120000121000122000123000124000125000126000127000128000129000130000131000132000133000134000135000136000137000138000139000140000141000142000143000144000145000146000147000148000149000150000151000152000153000154000155000156000157000158000159000160000161000162000163000164000165000166000167000168000169000170000171000172000173000174000175000176000177000178000179000180000181000182000183000184000185

In [9]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=X2, size=50, window=5, min_count=1, workers=4, sg=0)

In [10]:
model_result = model.wv.most_similar("▁man")
print(model_result)

[('▁woman', 0.926487147808075), ('▁lady', 0.8885421752929688), ('▁guy', 0.8851172924041748), ('▁gentleman', 0.8785795569419861), ('▁girl', 0.8782986402511597), ('▁worker', 0.8658108115196228), ('▁gentlemen', 0.8455085158348083), ('▁dude', 0.8162292242050171), ('▁bouncer', 0.7841111421585083), ('▁gal', 0.7780405879020691)]


In [11]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v2') # 모델 저장
#loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드